In [4]:
import cv2

import mediapipe as mp

#进度条库
from tqdm import tqdm

import time

##### 导入模型

In [5]:
# 导入三维人脸关键点检测模型
mp_face_mesh = mp.solutions.face_mesh
model = mp_face_mesh.FaceMesh(
    static_image_mode=False,       #是静态图片还是连续视频帧
    refine_landmarks=True,        #使用Attention Mesh模型，对嘴唇、眼睛、瞳孔周围的关键点精细定位
    max_num_faces=1,               #最多检测几张脸
    min_detection_confidence=0.5,  #置信度阈值，越接近1越准
    min_tracking_confidence=0.5    #追踪阈值
)

In [6]:
#导入可视化函数和可视化样式
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

##### 处理单帧的函数

In [7]:
# 处理函数
def process_frame(img):
    #记录该帧开始处理的时间
    start_time = time.time()
    scaler = 1 #文字大小
    #BGR转RGB
    img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # 将RGB图像输入模型 获取预测结果
    results = model.process(img_RGB)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            #绘制人脸网格
            mp_drawing.draw_landmarks(
                image = img,
                landmark_list = face_landmarks,
                connections = mp_face_mesh.FACEMESH_TESSELATION,
                #landmark_drawing_spec 为关键点可视化样式 None为默认值（不显示关键点）
                #landmark_drawing_spec= mp_drawing.DrawingSpec(thickness=1,circle_radius=2,color=[66,77,229])
                landmark_drawing_spec = None,
                connection_drawing_spec = mp_drawing_styles.get_default_face_mesh_tesselation_style()
            )
            #绘制脸轮廓、眼睫毛、眼眶、嘴唇
            mp_drawing.draw_landmarks(
                image = img,
                landmark_list = face_landmarks,
                connections = mp_face_mesh.FACEMESH_CONTOURS,
                #landmark_drawing_spec 为关键点可视化样式 None为默认值（不显示关键点）
                #landmark_drawing_spec= mp_drawing.DrawingSpec(thickness=1,circle_radius=2,color=[66,77,229])
                landmark_drawing_spec = None,
                connection_drawing_spec = mp_drawing_styles.get_default_face_mesh_contours_style()
            )
            #绘制瞳孔区域
            mp_drawing.draw_landmarks(
                image = img,
                landmark_list = face_landmarks,
                connections = mp_face_mesh.FACEMESH_IRISES,
                #landmark_drawing_spec 为关键点可视化样式 None为默认值（不显示关键点）
                #landmark_drawing_spec= mp_drawing.DrawingSpec(thickness=1,circle_radius=2,color=[66,77,229])
                landmark_drawing_spec = None,
                connection_drawing_spec = mp_drawing_styles.get_default_face_mesh_iris_connections_style()
            )
    else:
        img = cv2.putText(img,"No Face Detected",(25*scaler,50*scaler),cv2.FONT_HERSHEY_SIMPLEX,1.25*scaler,(255,0,255),2*scaler)

    #记录该帧处理完毕时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time-start_time)

    img = cv2.putText(img,'FPS  '+str(int(FPS)),(25*scaler,100*scaler),cv2.FONT_HERSHEY_SIMPLEX,1.25*scaler,(255,0,255),2*scaler)

    return img

##### 调用摄像头获取每帧（模板）

In [8]:
import cv2
import time
cap = cv2.VideoCapture(0)
cap.open(0)
#无限循环，知道break被触发
while cap.isOpened():
    success ,frame = cap.read()
    if not success:
        print('ERROR')
        break

    ### 处理帧函数
    frame = process_frame(frame)

    #展示处理后的三通道图像
    cv2.imshow('my_window',frame)
    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或者esc退出（在英文输入法下）
        break

# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

KeyboardInterrupt: 

##### 视频逐帧处理（模板）

In [9]:
def generate_video(input_path = 'data/dan.mp4'):
    #filehead = input_path.sp
    output_path = "out-dan.mp4"

    print('视频开始处理',input_path)

    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success,frame = cap.read()
        frame_count+=1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)

    cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH),cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path,fourcc,fps,(int(frame_size[0]),int(frame_size[1])))

    #进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success,frame = cap.read()
                if not success:
                    break
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                if success == True:
                    out.write(frame)

                    #进度条更新一帧
                    pbar.update(1)
        except:
            print('中途中断')
            pass
    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存',output_path)


In [10]:
generate_video(input_path='data/dan.mp4')

视频开始处理 data/dan.mp4
视频总帧数为 193


100%|██████████| 192/192 [00:09<00:00, 19.92it/s]


视频已保存 out-dan.mp4
